In [59]:
import networkx as nx
import random
from tqdm import tqdm

from typing import Tuple, Set, Any, List, Dict

In [84]:
G = nx.read_edgelist('./node2vec/graph/ca-AstroPh.edgelist', nodetype=int)

In [91]:
G_giant = G.subgraph(max(nx.connected_components(G), key=len)).copy()
n = G_giant.number_of_edges()//2

In [93]:
Node = Any

def random_remove_edges(G: nx.Graph, n: int) -> Set[Tuple[Node, Node]] :
    # inplace removal of random edges s.t. G stays connected
    edges = list(G_giant.edges())
    removed = set()
    i = 0
    while (i < n):
        (idx,) = random.sample(range(len(edges)), 1)
        (u, v) = edges.pop(idx)
        if not(G.degree(u) == 1) and not(G.degree(v) == 1) :
            G_giant.remove_edge(u, v)
            i += 1
            removed.add((u, v))
    return removed

In [94]:
removed = random_remove_edges(G_giant, n)

In [95]:
(G_giant.number_of_nodes() - len(removed)) <= 1

True

In [98]:
nx.write_edgelist(G_giant, './node2vec/graph/ca-AstroPh-half.edgelist')

In [102]:
with open('./node2vec/graph/ca-AstroPh-removed.edgelist', 'w') as handle : 
    handle.writelines(f'{u} {v}\n' for (u, v) in removed)

<hr>

In [9]:
import numpy as np

In [30]:
G = nx.read_edgelist('./node2vec/graph/ca-AstroPh-half.edgelist')

In [5]:
!python ./node2vec/src/main.py --input ./node2vec/graph/ca-AstroPh-half.edgelist --output ./node2vec/emb/ca-AstroPh-half.emb

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [31]:
id2idx = {id : idx for (idx, id) in enumerate(sorted(G.nodes()))}

In [32]:
with open('./node2vec/emb/ca-AstroPh-half.emb', 'r') as handle:
    (n, d) = map(int, handle.readline().strip().split())

    embeddings = np.empty((n, d), dtype=np.float32)
    while (line := handle.readline().strip()):
        (id, *emb) = line.split()
        # nodes start at 1 instead of 0.
        embeddings[id2idx[id]] = list(map(float, emb))

In [60]:
def diffuse_embeddings(G: nx.Graph, id2idx: Dict[str,int], X: List[List[float]], gamma=0.05) -> List[List[float]] :
    # diffusing the learned embeddings based on their neighbors
    deg_avg = sum(d for (n, d) in G.degree()) / G.number_of_nodes()

    Xprime = X.copy()
    for u in G.nodes() : 
        for v in G.neighbors(u) : 
            Xprime[id2idx[u]] += gamma*(G.degree(v)/deg_avg)*X[id2idx[v]]
    return Xprime

In [61]:
embeddings = diffuse_embeddings(G, id2idx, embeddings)

# Training Data

In [62]:
m = G.number_of_edges()

X_train = np.empty((2*m, embeddings.shape[1]), dtype=np.float32)
y_train = np.zeros(len(X_train), dtype=np.float32)
y_train[:m] = 1

# positive examples
for i, (u, v) in enumerate(G.edges()):
    X_train[i] = embeddings[id2idx[u]] * embeddings[id2idx[v]]
# negative examples
i = 0
nodes = list(G.nodes())
while (i < m) :
    (u, v) = random.sample(nodes, 2)
    if not(G.has_edge(u, v)) :
        X_train[m+i] = embeddings[id2idx[u]] * embeddings[id2idx[v]]
        i += 1

# Testing Data

In [63]:
with open('./node2vec/graph/ca-AstroPh-removed.edgelist', 'r') as handle:
    removed = {tuple(line.strip().split()) for line in handle}

In [64]:
m = len(removed)

X_test = np.empty((2*m, embeddings.shape[1]), dtype=np.float32)
y_test = np.zeros(len(X_test), dtype=np.float32)
y_test[:m] = 1

# positive examples
for i, (u, v) in enumerate(removed):
    X_test[i] = embeddings[id2idx[u]] * embeddings[id2idx[v]]
# negative examples
i = 0
nodes = list(G.nodes())
while (i < m) :
    (u, v) = random.sample(nodes, 2)
    if not(G.has_edge(u, v)) :
        X_test[m+i] = embeddings[id2idx[u]] * embeddings[id2idx[v]]
        i += 1

# Link Prediction

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

roc_auc_score(y_test, y_pred)

0.9055524539410242

In [67]:
clf.score(X_test, y_test)

0.9055524539410242